# Transition-based dependency parsing

In the [parsing chapter](parsing.ipynb) we saw how to develop a syntactic parser based on context-free grammars (CFG). In this chapter we will see how to develop a parser based on a different paradigm, dependency trees.

TODO: Show the matrix representation of the depedency graph, heads and modifiers

TODO: define the transition system with functions that are called showing the stack and buffer. Keep the edges generated in a list. Or is there a way of getting the dependency graph generated dynamically with python lib? networkx?

TODO: This blog post is quite good: https://spacy.io/blog/parsing-english-in-python

TODO: Nivre description

TODO: Sample transition system

TODO: Different transition systems: they consist of some elementary actions. The key thing is the learnability of the system: ambiguity is confusing.

TODO: Dynamic oracles to help with the ambiguity that we are left with, it was an implementation of the idea of expert policy from imitation learning (Searn/DAgger)

TODO: Non-projectivity with dual stacks